In [4]:
#%pip install git+https://github.com/jgieseler/solo-epd-loader --upgrade
#%pip install calc_electrons

In [5]:
from SOLO_loaders import solo_mag_loader
from solo_functions import calc_pa_coverage, calc_av_en_flux_EPD, calc_EPT_corrected_e, print_HET_energies, print_EPT_energies
from epd_plot_shift import *
from run_the_fit import *
#from solo_epd_loader import calc_electrons
from solo_epd_loader import epd_load, calc_electrons, combine_pixels # import via pip
import datetime as dt
from pandas.tseries.frequencies import to_offset
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
from matplotlib.ticker import AutoMinorLocator 
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['font.size'] = 10
plt.rcParams['agg.path.chunksize'] = 20000
import numpy as np
import pandas as pd
import sys


col_list = plt.cm.rainbow(np.linspace(0.,0.95,16))

/home/annafed/anaconda3/envs/annamaria/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
outpath = r'/home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/'
intensity_label = 'Flux\n/(s cm² sr MeV)'

save_fig = False
save = True # new variable in Anna's notebook

date = datetime(2022, 4, 15, 3, 2) #onset time or event ID inj time specified later
t_inj = str(date)[:-3].replace(' ', '-').replace(':', '')
date_folder = t_inj

year = 2022
startdate = dt.datetime(year, 4, 15, 5, 30)
enddate = dt.datetime(year, 4, 15, 16, 30)


mag_data_product = 'l2'# 'l2', 'll'
data_product = 'l2'
instrument = 'EPT'
species = 'e' # e or p
direction = 'sun'
ept_ion_conta_corr = True  # if true EPT electron fluxes are corrected for ion contamination
plot_pa_cov = True



av = 1  # needed for solo_mag_loader
averaging = f'{av}min' 

## EPT:
en_channel = 1
## en_channel: when a list of two elements is provided, an averaged energy channel is used (not yet supported for STEP!). 
## If only one element is provided, then only this one channel is used.

## STEP
step_en_channel = 29

# 2, 25 e en chan
# 1, 29 p en chan
#-----------------
if species in ['e', 'electrons']:
    step_species = 'Electron'
if species in ['p', 'i', 'protons', 'ions']:
    step_species = 'Magnet'



In [7]:
print(averaging)

1min


# LOAD DATA

In [8]:
path =  r'/home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/'
downloadpath = f'{path}l2/epd/{instrument}/'
sdate = int(f'{startdate.date()}'.replace('-', ''))
edate = int(f'{enddate.date()}'.replace('-', ''))



# MAG and PA coverage
print('Loading MAG...')
msdate = dt.datetime.combine(startdate.date(), dt.time.min)
medate = dt.datetime.combine(enddate.date()+ dt.timedelta(days=1), dt.time.min) #+ dt.timedelta(days=1)
enddate_epd = dt.datetime.combine(enddate.date(), dt.time.max)
try:
    mag_srf = solo_mag_loader(msdate, medate, level=mag_data_product, frame='srf', av=averaging, path=path)
except:
    print('changing mag data product to LL')
    mag_type = 'LL'
    mag_srf = solo_mag_loader(msdate, medate, level=mag_data_product,frame='srf', av=averaging, path=path)

mag_srf = mag_srf[mag_srf.index <= enddate_epd]

coverage = calc_pa_coverage(instrument, mag_srf)
coverage_step = calc_pa_coverage('step', mag_srf)




print('Loading STEP...')
df_particles_step_org, energies_step = epd_load('step', level=data_product, startdate=sdate, enddate=edate, path=path, autodownload=True)

print('Loading '+instrument+'...')
if instrument in ['EPT', 'HET']:
    df_sun_p, df_sun_e, energies = epd_load(instrument, level=data_product, startdate=sdate, enddate=edate, viewing='sun', path=path, autodownload=True)

Loading MAG...
Loading MAG...
Calculating PA coverage for EPT...


100%|██████████| 1440/1440 [00:00<00:00, 10116.68it/s]


Calculating PA coverage for step...
Opening of STEP just a placeholder! Replace with real value! This affects the 'min' and 'max' values of the pitch-angle, not the 'center' ones.


100%|██████████| 1440/1440 [00:00<00:00, 2810.11it/s]


Loading STEP...
Loading /home/annafed/Documents/STIX_EPD_event_analysis/STEP_EPD_comparison/l2/epd/step/solo_L2_epd-step-main_20220415_V01.cdf
Loading EPT...


In [9]:
#df_particles_step_org

## print energy channels

In [10]:
print_EPT_energies(energies)  # for HET: print_HET_energies(energies)


Ions
0 ['0.0495 - 0.0574 MeV']
1 ['0.0520 - 0.0602 MeV']
2 ['0.0552 - 0.0627 MeV']
3 ['0.0578 - 0.0651 MeV']
4 ['0.0608 - 0.0678 MeV']
5 ['0.0645 - 0.0718 MeV']
6 ['0.0689 - 0.0758 MeV']
7 ['0.0729 - 0.0798 MeV']
8 ['0.0768 - 0.0834 MeV']
9 ['0.0809 - 0.0870 MeV']
10 ['0.0870 - 0.0913 MeV']
11 ['0.0913 - 0.0974 MeV']
12 ['0.0974 - 0.1034 MeV']
13 ['0.1034 - 0.1096 MeV']
14 ['0.1096 - 0.1173 MeV']
15 ['0.1173 - 0.1246 MeV']
16 ['0.1246 - 0.1333 MeV']
17 ['0.1333 - 0.1419 MeV']
18 ['0.1419 - 0.1514 MeV']
19 ['0.1514 - 0.1628 MeV']
20 ['0.1628 - 0.1744 MeV']
21 ['0.1744 - 0.1879 MeV']
22 ['0.1879 - 0.2033 MeV']
23 ['0.2033 - 0.2189 MeV']
24 ['0.2189 - 0.2364 MeV']
25 ['0.2364 - 0.2549 MeV']
26 ['0.2549 - 0.2744 MeV']
27 ['0.2744 - 0.2980 MeV']
28 ['0.2980 - 0.3216 MeV']
29 ['0.3216 - 0.3494 MeV']
30 ['0.3494 - 0.3810 MeV']
31 ['0.3810 - 0.4117 MeV']
32 ['0.4117 - 0.4472 MeV']
33 ['0.4472 - 0.4850 MeV']
34 ['0.4850 - 0.5255 MeV']
35 ['0.5255 - 0.5734 MeV']
36 ['0.5734 - 0.6216 MeV']
37 ['

In [19]:
energies_step

{'Bins_Low_Energy': array([0.005725, 0.006103, 0.006521, 0.006998, 0.007548, 0.008154,
        0.008802, 0.009491, 0.01026 , 0.01109 , 0.01198 , 0.01297 ,
        0.01405 , 0.01516 , 0.01655 , 0.01786 , 0.01933 , 0.02092 ,
        0.02261 , 0.02446 , 0.02652 , 0.0287  , 0.03101 , 0.03346 ,
        0.03618 , 0.0394  , 0.04254 , 0.04589 , 0.04977 , 0.0539  ,
        0.05826 , 0.06288 ], dtype=float32),
 'Bins_Width': array([0.003233, 0.003021, 0.002852, 0.002754, 0.002682, 0.002626,
        0.002598, 0.002599, 0.00261 , 0.00261 , 0.0026  , 0.00269 ,
        0.00278 , 0.00288 , 0.00272 , 0.00272 , 0.00278 , 0.00279 ,
        0.00281 , 0.0029  , 0.00295 , 0.003   , 0.00305 , 0.00319 ,
        0.00322 , 0.00314 , 0.00335 , 0.00388 , 0.00413 , 0.00436 ,
        0.00462 , 0.00512 ], dtype=float32),
 'Bins_Text': array([['0.0057 - 0.0090 MeV'],
        ['0.0061 - 0.0091 MeV'],
        ['0.0065 - 0.0094 MeV'],
        ['0.0070 - 0.0098 MeV'],
        ['0.0075 - 0.0102 MeV'],
        ['0.0082 - 

# time averaging and EPT ion conta corr

In [9]:
if averaging is not None:
    coverage = coverage.resample(averaging,label='left').mean()
    coverage.index = coverage.index + to_offset(pd.Timedelta(averaging)/2)
    
    #if instrument == 'STEP':
    if species == 'e':
        contamination_threshold = 2
        df_step = calc_electrons(df=df_particles_step_org, meta=energies_step, 
                                   contamination_threshold=contamination_threshold, only_averages=False,
                                   resample=averaging)
        df_step_inner = combine_pixels(df=df_step, meta=energies_step, pixels=['02', '03', '04', '07', '08', '09', '12', '13', '14'])
    else:
        None
        # to do: implement ion fluxes
        #df_sun = df_particles_step_org  
        
        
    if instrument in ['EPT', 'HET']:
        df_sun_p   = df_sun_p.resample(averaging,label='left').mean()
        df_sun_p.index = df_sun_p.index + to_offset(pd.Timedelta(averaging)/2)
    
    if instrument in ['EPT', 'HET']:
        df_sun_e   = df_sun_e.resample(averaging,label='left').mean()
        df_sun_e.index = df_sun_e.index + to_offset(pd.Timedelta(averaging)/2)

if instrument=='EPT':
    if ept_ion_conta_corr:
        df_sun_e = calc_EPT_corrected_e(df_sun_e['Electron_Flux'], df_sun_p['Ion_Flux'])




In [10]:
#df_step_inner['Electron_Comb_Flux_22']
#df_step_inner

In [11]:
df_sun_p

Ion_Flux                                         \
                    Ion_Flux_0   Ion_Flux_1   Ion_Flux_2   Ion_Flux_3   
Time                                                                    
2022-04-15 00:00:30        NaN  1379.302734  1030.180176   797.692444   
2022-04-15 00:01:30        NaN  1379.302734   515.090088   797.692444   
2022-04-15 00:02:30        NaN  1683.216919     0.000000   811.212646   
2022-04-15 00:03:30        NaN  1379.302734   515.090088  1196.538696   
2022-04-15 00:04:30        NaN   827.581665  1030.180176   797.692444   
...                        ...          ...          ...          ...   
2022-04-15 23:55:30        NaN  1655.163330     0.000000  1994.231079   
2022-04-15 23:56:30        NaN  1931.023804  1545.270142  1994.231079   
2022-04-15 23:57:30        NaN  1103.442139  1545.270142   797.692444   
2022-04-15 23:58:30        NaN  1103.442139  2060.360352   398.846222   
2022-04-15 23:59:30        NaN  1655.163330   515.090088   398.846222   

                                                                         \
                      Ion_Flux_4   Ion_Flux_5   Ion_Flux_6   Ion_Flux_7   
Time                                                                      
2022-04-15 00:00:30   671.636780   234.889999     0.000000     0.000000   
2022-04-15 00:01:30   335.818390     0.000000     0.000000   261.825623   
2022-04-15 00:02:30  1366.040894   238.871201     0.000000     0.000000   
2022-04-15 00:03:30     0.000000   469.779999   276.601288     0.000000   
2022-04-15 00:04:30   335.818390   469.779999   276.601288   261.825623   
...                          ...          ...          ...          ...   
2022-04-15 23:55:30  2014.910400   939.559998  1659.607788  1047.302490   
2022-04-15 23:56:30  1679.091919   704.670044   553.202576  2094.604980   
2022-04-15 23:57:30  1679.091919  1409.340088   829.803894  2094.604980   
2022-04-15 23:58:30   671.636780  2114.010010  1383.006470  1309.128174   
2022-04-15 23:59:30  1343.273560  2114.010010  1383.006470  1832.779297   

                                               ...   Alpha_Rate               \
                      Ion_Flux_8   Ion_Flux_9  ... Alpha_Rate_1 Alpha_Rate_2   
Time                                           ...                             
2022-04-15 00:00:30     0.000000     0.000000  ...          0.0          0.0   
2022-04-15 00:01:30   959.197632   209.828354  ...          0.0          0.0   
2022-04-15 00:02:30     0.000000     0.000000  ...          0.0          0.0   
2022-04-15 00:03:30   319.732544   629.485046  ...          0.0          0.0   
2022-04-15 00:04:30   319.732544     0.000000  ...          0.0          0.0   
...                          ...          ...  ...          ...          ...   
2022-04-15 23:55:30   639.465088   629.485046  ...          0.0          0.0   
2022-04-15 23:56:30   959.197632   419.656708  ...          0.0          0.0   
2022-04-15 23:57:30  1598.662720  1468.798462  ...          0.0          0.0   
2022-04-15 23:58:30  1918.395264  1049.141846  ...          0.0          0.0   
2022-04-15 23:59:30   959.197632  1888.455200  ...          0.0          0.0   

                                                                         \
                    Alpha_Rate_3 Alpha_Rate_4 Alpha_Rate_5 Alpha_Rate_6   
Time                                                                      
2022-04-15 00:00:30          0.0          0.0          0.0     0.000000   
2022-04-15 00:01:30          0.0          0.0          0.0     0.000000   
2022-04-15 00:02:30          0.0          0.0          0.0     0.000000   
2022-04-15 00:03:30          0.0          0.0          0.0     0.000000   
2022-04-15 00:04:30          0.0          0.0          0.0     0.000000   
...                          ...          ...          ...          ...   
2022-04-15 23:55:30          0.0          0.0          0.0     0.000000   
2022-04-15 23:56:30          0.0          0.0          0.0     0.016667   
20

In [12]:
df_sun.keys

NameError: name 'df_sun' is not defined

In [13]:
if species in ['e', 'electrons']:
    if instrument in ['EPT', 'HET']:
        df_sun = df_sun_e
    en_str = energies['Electron_Bins_Text']
    ch_string = 'Electrons'
    if instrument == 'EPT':
        if ept_ion_conta_corr:
            ch_string = 'Electrons (corr)'
    
if species in ['p', 'i', 'protons', 'ions']:
    if instrument in ['EPT', 'HET']:
        df_sun = df_sun_p
    if instrument in ['STEP', 'EPT']: 
        en_str = energies['Ion_Bins_Text']
        ch_string = 'Ions'
    if instrument == 'HET': 
        en_str = energies['H_Bins_Text']
        ch_string = 'Protons'



######################################################################################
#if instrument == 'STEP':
if species in ['e', 'electrons']:
    en_channel_string_step = energies_step['Electron_Bins_Text'][step_en_channel][0]
if species in ['p', 'i', 'protons', 'ions']:
    en_channel_string_step = energies_step['Bins_Text'][step_en_channel]

# EPT / HET    
I_sun, en_channel_string = calc_av_en_flux_EPD(df_sun, energies, en_channel, species, instrument)




KeyError: 'Ion_Flux_1'

# MAKE THE PLOT

In [ ]:
#startdate = dt.datetime(year, 10, 28, 15, 30)
#enddate = dt.datetime(year, 10, 28, 17, 30)

particle = ''
if species = 'e':
    particle = 'electron'
else:
    particle = 'proton'

In [ ]:
%matplotlib notebook
panels = 1
if plot_pa_cov:
    panels = panels +1 
if panels <=2:
    figsize = (8,7)
if 2 < panels <=5:
    figsize = (8,5)
if panels > 5:
    figsize = (7,9)
fig, axes = plt.subplots(panels, figsize=figsize, sharex=True) #figsize=(9,7)
#color = {'sun':'crimson','asun':'orange', 'north':'darkslateblue', 'south':'c'}
color = {'sun':'gray'}
axnum = 0

plist = [5,10,15,4,9,14,3,8,13,2,7,12,1,6,11]  # STEP pixel list


ax = axes[axnum]
if instrument in ['HET', 'EPT']:    
    flux_dic = {'sun': I_sun}#, 'asun':I_asun, 'north':I_north, 'south':I_south}

if plot_pa_cov:
    # Pitch angle coverage:
    ####################################################################
    # instrument == 'STEP':
    for i in range(15):
        p = plist[i]
        ax.plot(coverage_step.index, coverage_step[f'Pixel_{p}']['center'], color = col_list[i], 
                linewidth=1, label=f'Pixel_{p}', drawstyle='steps-mid')
    if instrument in ['HET', 'EPT']: 
        #for direction in ['sun', 'asun', 'north', 'south']: 
        col = color[direction]
        # fill the minimum-maximum range of the pitch angle coverage
        ax.fill_between(coverage.index, coverage[direction]['min'], coverage[direction]['max'], alpha=0.5, color=col, edgecolor=col, linewidth=0.0, step='mid')
        # plot the central pitch angle as a thin line
        ax.plot(coverage.index, coverage[direction]['center'], linewidth=0.7, label=direction, color=col, drawstyle='steps-mid')



    ax.axhline(y=90, color='gray', linewidth=0.8, linestyle='--')
    ax.axhline(y=45, color='gray', linewidth=0.8, linestyle='--')
    ax.axhline(y=135, color='gray', linewidth=0.8, linestyle='--')


    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title=instrument+ ' & STEP')
    ax.set_ylim([0, 180])
    ax.yaxis.set_ticks(np.arange(0, 180+45, 45))
    ax.set_xlim([coverage.index[0], coverage.index[-1]])
    ax.set_ylabel('Pitch angle / °')
    ax.tick_params(axis="x",direction="in", which='both', pad=-15)
    ax.tick_params(labelbottom=False, labeltop=False, labelleft=True, labelright=False, bottom=True, top=True, left=False, right=False)
    axnum = axnum + 1
    

# Intensities
if instrument in ['HET', 'EPT']: 
    ax = axes[axnum]
    av_flux = flux_dic[direction]
    ax.plot(av_flux.index.values, av_flux.values, linewidth=1, label=instrument+' '+en_channel_string, color='k', drawstyle='steps-mid')

# instrument == 'STEP':
for i in range(15):
    p = plist[i]
    ax = axes[axnum]
    ax.plot(df_step.index, df_step[f'{step_species}_{p:02d}_Flux_{step_en_channel}'].values, color = col_list[i],
            linewidth=0.7, drawstyle='steps-mid')

    
ax.plot(df_step_inner.index, df_step_inner[f'{step_species}_Comb_Flux_{step_en_channel}'].values, color = 'magenta',
            linewidth=1, drawstyle='steps-mid')
    
    
# this is just for the step energy label:    
p=1
ax.plot([np.nan], [np.nan], color = col_list[i],
        linewidth=0.7, drawstyle='steps-mid', label='STEP '+en_channel_string_step)


ax.text(0.05, 0.94, en_channel_string, horizontalalignment='left', verticalalignment='top', transform = ax.transAxes)
ax.set_yscale('log')
ax.set_ylabel(intensity_label)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.tick_params(axis="x",direction="in", which='both')#, pad=-15)
axnum = axnum + 1




ax.set_xlim(startdate, enddate)
#########################
ax = axes[-1]
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d\n%H:%M"))
ax.xaxis.set_minor_locator(AutoMinorLocator(2))
ax.set_xlabel('Date / Time in year '+str(year))
axes[0].set_title(f'Solar Orbiter EPT & STEP')




plt.tight_layout()
fig.subplots_adjust(hspace=0.1)



if save_fig:
    en_ch_str = str(en_channel).replace('[', '')
    en_ch_str = en_ch_str.replace(']', '')
    en_ch_str = en_ch_str.replace(' ', '')
    en_ch_str = en_ch_str.replace(',', '-')
    plt.savefig(f'{outpath}{instrument}_EPT_vs_STEP_Pixels_{str(sdate)}_{species}_ch{en_ch_str}_{averaging}.png')
    plt.close()
else:
    plt.show()
    
if save:
    plt.savefig(path +date_folder+ '/plots/STEP-EPT-comparison-PA-'+direction+'-'+particle+'-'+averaging+'.png', dpi=300, bbox_inches='tight')


In [ ]:
#%matplotlib notebook
f, axes = plt.subplots(5,3, figsize=(10, 15), sharex=True)
p=0
plist = [5,10,15,4,9,14,3,8,13,2,7,12,1,6,11]
for i, ax in enumerate(axes.flat):
    p=plist[i]
    av_flux = flux_dic[direction]
    ax.plot(av_flux.index.values, av_flux.values, linewidth=1, color='k', drawstyle='steps-mid')
    ax.plot(df_step.index, df_step[f'{step_species}_{p:02d}_Flux_{step_en_channel}'].values, color = col_list[i],
            linewidth=0.7, drawstyle='steps-mid',label=f'Pixel_{p}')
    ax.set_yscale('log')
    ax.legend()
    plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
ax.set_xlim(startdate, enddate)
axes.flat[1].set_title(f'EPT ({en_channel_string}) vs. STEP ({en_channel_string_step})')
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
#plt.savefig(path +date_folder+ '/plots/STEP-EPT-comparison-PA-'+direction+'-'+averaging+'.png', dpi=300, bbox_inches='tight')

if save:
    plt.savefig(path +date_folder+ '/plots/STEP-EPT-comparison-pixels-intensity-'+direction+'-'+particle+'-'+averaging+'.png')#, dpi=300, bbox_inches='tight')



In [ ]:
print('STEP-EPT-comparison-PA-'+direction+'-'+averaging)
print('STEP-EPT-comparison-pixels-intensity-'+direction+'-'+averaging)